<a href="https://colab.research.google.com/github/naumovdk/recommender-systems/blob/master/HomeWorks/hw1-matrix-factorization/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [3]:
# !pip install implicit
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

# from lightfm.datasets import fetch_movielens

     |████████████████████████████████| 1.1MB 19.3MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406403 sha256=6403ef999e4f08326a358b3c8be12e0feeb598c4d3064cc9b53ad5d829fdbec1
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [4]:
github_ratings = 'https://raw.githubusercontent.com/naumovdk/recommender-systems/master/HomeWorks/hw1-matrix-factorization/ml-1m/ratings.dat'
ratings = pd.read_csv(github_ratings, delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [5]:
github_movies = 'https://raw.githubusercontent.com/naumovdk/recommender-systems/master/HomeWorks/hw1-matrix-factorization/ml-1m/movies.dat'
movie_info = pd.read_csv(github_movies, delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding='ISO-8859-1')

Explicit данные

In [6]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [7]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [8]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [10]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [12]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [13]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [15]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '1526    Hercules (1997)',
 '2692    Iron Giant, The (1999)',
 '2252    Pleasantville (1998)',
 '3817    Went to Coney Island on a Mission From God... ...']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [16]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [17]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [18]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [19]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1884    French Connection, The (1971)',
 '1179    Princess Bride, The (1987)',
 '1892    Rain Man (1988)',
 '3458    Predator (1987)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [20]:
from sklearn.base import TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split

In [21]:
class SGD:
    def __init__(self, features=40, iterations=10_000, learning_rate=0.01, matrix_reg=0.01, bias_reg=0.01, scale=0.1, verbose=1):
        self.features = features
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.scale = np.power(scale, 1 / iterations)
        
        self.user_reg = matrix_reg
        self.item_reg = matrix_reg
        self.user_bias_reg = bias_reg
        self.item_bias_reg = bias_reg

        self.verbose = verbose


    def fit(self, ratings=ratings):
        self.users = ratings['user_id'].max()
        self.items = ratings['movie_id'].max()
        self.avg = ratings['rating'].mean()

        self.U = np.random.uniform(0, 1 / np.sqrt(self.features), (self.users, self.features))
        self.I = np.random.uniform(0, 1 / np.sqrt(self.features), (self.items, self.features)) 

        self.user_bias = np.zeros((self.users, 1))
        self.item_bias = np.zeros((self.items, 1))
    
        for iter in range(self.iterations):
            i, j, actual = map(int, ratings.sample().to_numpy().ravel())
            i -= 1
            j -= 1
            
            error = (self.U[i] @ self.I[j].T + self.user_bias[i] + self.item_bias[j].T + self.avg) - actual
            
            self.U[i] -= self.learning_rate * (error * self.I[j]   + self.user_reg * self.U[i])
            self.I[j] -= self.learning_rate * (error * self.U[i].T + self.item_reg * self.I[j])
            self.user_bias[i] -= self.learning_rate * (error + self.user_bias_reg * self.user_bias[i])
            self.item_bias[j] -= self.learning_rate * (error + self.item_bias_reg * self.item_bias[j])

            self.learning_rate *= self.scale

            if iter % (self.iterations // self.verbose) == 0:
                rmse = np.linalg.norm(ratings['rating'] - self.predict()[ratings['user_id'] - 1, ratings['movie_id'] - 1])
                rmse /= len(ratings)
                print(f'rmse {rmse}')
                print(f'step {self.learning_rate}')
                print()
        
        return self


    def predict(self):
        return self.U @ self.I.T + self.user_bias + self.item_bias.T + self.avg

    
    def similar_items(self, i, n=10):
        metric = lambda j: np.linalg.norm(self.I[i] - self.I[j])
        distances = [(j, metric(j)) for j in range(len(self.I))]
        return sorted(distances, key=lambda pair: pair[1])[:n]

In [22]:
%%time
sgd = SGD(iterations=400_000, verbose=4).fit()

rmse 0.0011449427423857541
step 0.00999994243553836

rmse 0.001021403069384898
step 0.005623380881034522

rmse 0.0009931892865256587
step 0.0031622594566947933

rmse 0.0009821093304909836
step 0.0017782691734755693

CPU times: user 2h 4min 39s, sys: 27.6 s, total: 2h 5min 7s
Wall time: 2h 5min 4s


In [23]:
get_similars(1, sgd)

['0    Toy Story (1995)',
 '2344    Clue (1985)',
 '1040    Get on the Bus (1996)',
 'Series([], )',
 '171    Judge Dredd (1995)',
 '1545    G.I. Jane (1997)',
 '2348    Heartburn (1986)',
 '2382    Gate, The (1987)',
 '1946    Absent Minded Professor, The (1961)',
 '3393    Modern Times (1936)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [ ]:
# class ALS:
#     def __init__(self, features=40, iterations=100, learning_rate=0.001, reg=1):
#         self.features = features
#         self.iterations = iterations
#         self.learning_rate = learning_rate
#         self.reg = reg

#     def fit(self, ratings=implicit_ratings):
#         self.ratings = ratings
#         self.users = self.ratings['user_id'].max()
#         self.items = self.ratings['movie_id'].max()
        
#         self.U = np.random.uniform(0, 1 / np.sqrt(self.features), (self.users, self.features))
#         self.I = np.random.uniform(0, 1 / np.sqrt(self.features), (self.items, self.features))

#         for iteration in range(self.iterations):
#             error = self.U @ self.I.T - user_item_csr


# ALS().fit()

In [ ]:
implicit_ratings
user_item_csr[1, 3408]

1

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных